In [5]:
spark.sql(f"DROP DATABASE IF EXISTS silver CASCADE")
spark.sql(f"CREATE DATABASE IF NOT EXISTS silver") 

DataFrame[]

In [ ]:
update_audit_table("EV_Data", progress="Ingest_Silver", status="In_Progress", start=True)

In [ ]:
from pyspark.sql.functions import when, col, regexp_replace, trim

# Step 1: Define your database and table names
bronze_database_name = "bronze"
columns_metadata_table = "columns_metadata"
silver_database_name = "silver"
new_table_name = "ev_std"

# Step 2: Fetch the schema of your original table (columns metadata)
df = spark.sql(f"SELECT name, dataTypeName FROM {bronze_database_name}.{columns_metadata_table}")
df.limit(10).show()

# Step 3: Apply the mapping to convert your custom dataTypeName to actual SQL data types
df_with_sql_types = df.withColumn(
    "SQL_DataType",
    when(df.dataTypeName == "number", "DOUBLE")
    .when(df.dataTypeName == "text", "STRING")
    .when(df.dataTypeName == "point", "STRING")
    .when(df.dataTypeName == "meta_data", "STRING")
    .otherwise("UNKNOWN")
)

# Step 4: Correct only the 'name' column by replacing spaces and dashes with underscores using regexp_replace
df_with_sql_types = df_with_sql_types.withColumn(
    "name",
    regexp_replace(col("name"), "[^a-zA-Z0-9]+", "_")
)
df_with_sql_types = df_with_sql_types.withColumn(
    "name",
    trim(regexp_replace(col("name"), "^_+|_+$", ""))
)

# Step 5: Display the corrected schema with column names and data types
df_with_sql_types.limit(10).show()

# Step 6: Generate the CREATE TABLE statement based on the corrected schema
columns_with_types = [
    f"`{row['name']}` {row['SQL_DataType']}" for row in df_with_sql_types.collect()
]

create_table_statement = f"""
CREATE TABLE if not exists {silver_database_name}.{new_table_name} (
    {', '.join(columns_with_types)}
) USING DELTA 
"""

# Step 7: Print the CREATE TABLE statement for verification
print(create_table_statement)

# Execute the SQL statement to create the new table
spark.sql(create_table_statement)

# Confirm the table has been created successfully
print(f"Table {silver_database_name}.{new_table_name} has been created successfully.")


In [ ]:
def data_type_correction(bronze_df):
    
    bronze_df = bronze_df.withColumn("created_at", bronze_df["created_at"].cast(DoubleType()))\
                        .withColumn("updated_at", bronze_df["updated_at"].cast(DoubleType()))\
                        .withColumn("Postal_Code", bronze_df["Postal_code"].cast(DoubleType()))\
                        .withColumn("Model_year", bronze_df["Model_year"].cast(DoubleType()))\
                        .withColumn("2020_Census_Tract", bronze_df["2020_Census_Tract"].cast(DoubleType()))\
                        .withColumn("Counties", bronze_df["Counties"].cast(DoubleType()))\
                        .withColumn("Electric_Range", bronze_df["Electric_Range"].cast(DoubleType()))\
                        .withColumn("Base_MSRP", bronze_df["Base_MSRP"].cast(DoubleType()))\
                        .withColumn("Legislative_District", bronze_df["Legislative_District"].cast(DoubleType()))\
                        .withColumn("DOL_Vehicle_ID", bronze_df["DOL_Vehicle_ID"].cast(DoubleType()))\
                        .withColumn("Congressional_Districts", bronze_df["Congressional_Districts"].cast(DoubleType()))\
                        .withColumn("WAOFM_GIS_Legislative_District_Boundary", bronze_df["WAOFM_GIS_Legislative_District_Boundary"].cast(DoubleType()))
    bronze_df = bronze_df.withColumn("created_at",to_timestamp(from_unixtime(col("created_at"))))\
                           .withColumn("updated_at",to_timestamp(from_unixtime(col("updated_at"))))

    return bronze_df

In [ ]:
def us_state_validation(bronze_df):

    valid_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 
                'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 
                'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 
                'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
    bronze_df = bronze_df.withColumn("err_code", when(~col("State").isin(valid_states), concat(col("err_code"), lit("DQ01"))).otherwise(col("err_code")))\
    .withColumn("err_message", when(~col("State").isin(valid_states), concat(col("err_message"), lit("Invalid Sate"))).otherwise(col("err_message")))
    bronze_df = bronze_df.withColumn("err_flg", when(col("err_code") != "", "Y").otherwise("N"))
    return bronze_df

In [ ]:
def null_check(bronze_df):
    # List of columns to check for null values, excluding the specified ones
    columns = [col_name for col_name in bronze_df.columns if col_name not in ["err_code", "err_message", "updated_meta", "created_meta"]]

    # Initialize null check for all columns together, so we only add error once
    null_condition = None

    for column in columns:
        # Create a condition to check if any of the columns has a null value
        column_null_condition = col(column).isNull()
        if null_condition is None:
            null_condition = column_null_condition
        else:
            null_condition = null_condition | column_null_condition

    # Apply the error code and message only once if any column has a null value
    bronze_df = bronze_df.withColumn(
        "err_code",
        when((instr(col("err_code"), "DQ02") == 0) & null_condition, 
             concat(col("err_code"), lit("DQ02 "))
        ).otherwise(col("err_code"))
    ).withColumn(
        "err_message",
        when((instr(col("err_message"), "Null value") == 0) & null_condition, 
             concat(col("err_message"), lit("Null value "))
        ).otherwise(col("err_message"))
    )
    
    # Adding a final flag to indicate if there are any errors
    bronze_df = bronze_df.withColumn("err_flg", when(col("err_code") != "", lit("Y")).otherwise(lit("N")))

    return bronze_df


In [ ]:
def numeric_check(bronze_df):
    # List of numeric columns only
    numeric_columns = [field.name for field in bronze_df.schema.fields if field.dataType.simpleString() in ['int', 'double', 'float', 'long', 'decimal']]

    # Initialize numeric check for all columns together, so we only add error once
    invalid_numeric_condition = None

    for column in numeric_columns:
        # Create a condition to check if any of the columns has an invalid numeric value (NaN)
        column_invalid_condition = isnan(col(column))  # Check only for NaN (exclude nulls)

        # Combine conditions across all numeric columns
        if invalid_numeric_condition is None:
            invalid_numeric_condition = column_invalid_condition
        else:
            invalid_numeric_condition = invalid_numeric_condition | column_invalid_condition

    # Apply the error code and message only once if any column has an invalid numeric value
    bronze_df = bronze_df.withColumn(
        "err_code",
        when((instr(col("err_code"), "DQ04") == 0) & invalid_numeric_condition, 
             concat(col("err_code"), lit("DQ04 "))
        ).otherwise(col("err_code"))
    ).withColumn(
        "err_message",
        when((instr(col("err_message"), "Invalid Numeric Value") == 0) & invalid_numeric_condition, 
             concat(col("err_message"), lit("Invalid Numeric Value "))
        ).otherwise(col("err_message"))
    )

    # Adding a final flag to indicate if there are any errors
    bronze_df = bronze_df.withColumn("err_flg", when(col("err_code") != "", lit("Y")).otherwise(lit("N")))

    return bronze_df

In [ ]:
def year_check(bronze_df):
    current_year = datetime.now().year
    
    # Corrected the parentheses to ensure correct logical operation order
    bronze_df = bronze_df.withColumn(
        "err_code", 
        when((col("Model_year") <= current_year) & (col("Model_year") < 1900),
             concat(col("err_code"), lit("DQ03 "))
        ).otherwise(col("err_code"))
    )
    
    bronze_df = bronze_df.withColumn(
        "err_message", 
        when((col("Model_year") <= current_year) & (col("Model_year") < 1900),
             concat(col("err_message"), lit("Invalid Model Year "))
        ).otherwise(col("err_message"))
    )

    bronze_df = bronze_df.withColumn("err_flg", when(col("err_code") != "", lit("Y")).otherwise(lit("N")))
    
    return bronze_df

In [ ]:
from pyspark.sql.functions import col, when, concat, lit, instr, isnan, to_timestamp, from_unixtime
from pyspark.sql.types import TimestampType, DoubleType
from datetime import datetime

source_database_name = "bronze"
source_table_name = "vehicle_data" 
bronze_df = spark.sql(f"SELECT * FROM {source_database_name}.{source_table_name}")
bronze_df = bronze_df.withColumn("err_code", lit("")).withColumn("err_message", lit(""))
bronze_df = data_type_correction(bronze_df)
bronze_df = null_check(bronze_df)
bronze_df = year_check(bronze_df)
bronze_df = numeric_check(bronze_df)
# Create the table by writing data to it
bronze_df.write.format("delta").mode("overwrite").saveAsTable("silver.vehicle_data")
bronze_df.limit(10).show()

In [ ]:
update_audit_table("EV_Data", progress="Ingest_Silver", status="Completed", end=True)